<a href="https://colab.research.google.com/github/theQuert/inlpfun/blob/master/Demo/summary_parser_MDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip3 install transformers
import re
import urllib
from urllib import request
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

In [2]:
#@title Keywords input
urls = []
keywords_1 = 'covid covidvariant new variant south' #@param {type: "string"}
keywords_1 = re.sub(' ', '+', keywords_1)
start_date_1 = '2021-12-01' #@param {type: "string"}
end_date_1 = '2021-12-02' #@param {type: "string"}
search_inputs_1 = f'{keywords_1}+after%3A{start_date_1}+before%3A{end_date_1}'
url_1 = f"https://www.google.com/search?q={search_inputs_1}&tbm=nws&lr=lang_en&hl=en&sort=date&num=19"
urls.append(url_1)

keywords_2 = 'covid covidvaccine variant covidvariant new' #@param {type: "string"}
keywords_2 = re.sub(' ', '+', keywords_2)
start_date_2 = '2021-12-04' #@param {type: "string"}
end_date_2 = '2021-12-05' #@param {type: "string"}
search_inputs_2 = f'{keywords_2}+after%3A{start_date_2}+before%3A{end_date_2}'
url_2 = f"https://www.google.com/search?q={search_inputs_2}&tbm=nws&lr=lang_en&hl=en&sort=date&num=19"
urls.append(url_2)

keywords_3 = 'case confirmed florida first ireland' #@param {type: "string"}
keywords_3 = re.sub(' ', '+', keywords_3)
start_date_3 = '2021-12-07' #@param {type: "string"}
end_date_3 = '2021-12-09' #@param {type: "string"}
search_inputs_3 = f'{keywords_3}+after%3A{start_date_3}+before%3A{end_date_3}'
url_3 = f"https://www.google.com/search?q={search_inputs_3}&tbm=nws&lr=lang_en&hl=en&sort=date&num=19"
urls.append(url_3)

keywords_4 = 'covid covidvariant advised leak javid' #@param {type: "string"}
keywords_4 = re.sub(' ', '+', keywords_4)
start_date_4 = '2021-12-10' #@param {type: "string"}
end_date_4 = '2021-12-11' #@param {type: "string"}
search_inputs_4 = f'{keywords_4}+after%3A{start_date_4}+before%3A{end_date_4}'
url_4 = f"https://www.google.com/search?q={search_inputs_4}&tbm=nws&lr=lang_en&hl=en&sort=date&num=19"
urls.append(url_4)

keywords_5 = 'covid covidvariant javid uk sajid' #@param {type: "string"}
keywords_5 = re.sub(' ', '+', keywords_5)
start_date_5 = '2021-12-13' #@param {type: "string"}
end_date_5 = '2021-12-15' #@param {type: "string"}
search_inputs_5 = f'{keywords_5}+after%3A{start_date_5}+before%3A{end_date_5}'
url_5 = f"https://www.google.com/search?q={search_inputs_5}&tbm=nws&lr=lang_en&hl=en&sort=date&num=19"
urls.append(url_5)

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
    'cookie': 'ANID=AHWqTUluW4edOtDZ7-3F_53MFqdDN5T1DN-BNC-VcXXBw5C-KZsKjYGVJuCyD8kx; CONSENT=YES+TW.zh-TW+202002; OGPC=19022622-1:; SID=FQjW04nu_23-53Z_Lj1ZPwh5znV47FRwCmN_qzj9TRwSosFBXkUIP64k0Y-dUwDAz3mG_Q.; __Secure-1PSID=FQjW04nu_23-53Z_Lj1ZPwh5znV47FRwCmN_qzj9TRwSosFBHb5LI_6CXhDVlrIsgPpy6A.; __Secure-3PSID=FQjW04nu_23-53Z_Lj1ZPwh5znV47FRwCmN_qzj9TRwSosFBArXzwnhWV3VwJBhI1LX5lA.; HSID=AKipmF7Ju8gkn3bE4; SSID=ArMVTx4XoOf21X6Zi; APISID=gNBnak_4bGHBxuBb/AHWCwhVzktsMH3M-2; SAPISID=0x2XVB2tadnlG49F/AKdZkTW2zr8T-jcOp; __Secure-1PAPISID=0x2XVB2tadnlG49F/AKdZkTW2zr8T-jcOp; __Secure-3PAPISID=0x2XVB2tadnlG49F/AKdZkTW2zr8T-jcOp; NID=511=WZSeKnCToUmsTsi3o2LIkNHwJjIxkfqh2KX79ufabKT8GGv218Dr9FGso5W5O_LyoYkoPomabJnGKGEzuPjlRs15iQUi_NeAA9lXqWqR17PO074BOeqAKirMnIM4QroKm-N1F7t_J2IhE4dHn1jQ6swJkGxCaUJFjM9G1osjzJc9SMdiIy6UBtj4UXcnS4hWkyMOMiBNyLTRMBaH_UeaRLXD1ZSTr0ZwFz1XMYf7LhzZDIlNqAE-56UWOnN4vXAPoXH6zpkvigNULs4K3iBSuTP1d0BCbTC2ZPIuLe6aMDP_VisnVSw_3tG50Ku0qWXMWQjGW7ULqejQbAcHP3FLrlMvPMUqUT4poBHH7gQ8LZyUQZvDQzwalX9mOcDd3DvMVZlw4t7gpWV4w9unkYEWyx2aUKSJl9sN9gB5uJ5BB2AVcd76NrO854aOx4j6sSCrE1s0lPAUEc3OSwIKlvH_-Jnw36-eWUeS7sMnVeHJR_g2gx2bKnKfrDgKmbVh2tpOdq8riRg98ghPzlLrY20_Vvk2_Kd0SCaS3Z3Pk0KhY2ZbTibSV1qFjJsCM47iSumi5equJidqGA9jzAZNOxfdsjX106r21FZFT53ErQ; 1P_JAR=2022-01-08-02; SEARCH_SAMESITE=CgQIuJQB; SIDCC=AJi4QfEdlDP8JWptpn5Ei1jSlnwR6ov5CucFtcPRFKCruGaxc7du0w9afugi2PjtLf65kQpESkc; __Secure-3PSIDCC=AJi4QfEjT-vmOyI28lEJFAH71EEp8sRG2rSzyXGB-5o7Mw3LCumoNsvZsPjCJNf4uzGx7xc9HRM' 
}

In [24]:
summarization = []
def scrape(num, url):
    num = num
    url_list = []
    metas = []
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.content, "html.parser")
    soup_prettify = soup.prettify()
    with open(f'url_{num}.html', 'w') as fwrite:
        fwrite.write(soup_prettify)
    for link in soup.find_all('a'):
        url_list.append(link.get('href'))
    for url in url_list:
        if bool(re.findall('google.com', str(url)))==True:
            url_list.remove(url)
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    for url in url_list:
        if bool(re.findall(regex, str(url)))==True:
            metas.append(url)
    for meta in metas:
        if bool(re.findall('policies.google.com', str(meta)))==True:
            metas.remove(meta)
    '''
    for id in range(len(metas)):
        print(metas[id])

    '''
    inputs = []
    for meta in metas:
        try:
            resp = requests.get(meta, headers=headers, timeout=5)
        except requests.exceptions.ReadTimeout:
            print("READ TIMED OUT")
        soup = BeautifulSoup(resp.content, "html.parser")
        try:
            title = soup.title.string
            title_filtered = re.sub(r'(\s)-(\s)(\w+).+', '', title)
            title_filtered = title_filtered.strip('\r\n\t')
            inputs.append(title_filtered)
        except:
            print('Title convertion failed')
    try:
        summarizer = pipeline("summarization")
        summ=' '.join([str(i) for i in inputs])
        summarized = summarizer(summ, min_length=5, max_length=20)
        summarization.append(summarized[0]['summary_text'])
    except:
        print('Title conversion failed, summarization fails, too.')
    return metas

In [4]:
for id in range(len(urls)):
    scrape(id, urls[id])

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

READ TIMED OUT
READ TIMED OUT


In [25]:
# Print the meta urls of the first time duration
scrape(0, urls[0])

https://www.cnbc.com/2021/12/01/us-confirms-nations-first-case-of-omicron-covid-variant-in-california.html
https://www.nbcchicago.com/news/coronavirus/what-we-know-so-far-as-omicron-variant-identified-in-us/2697512/
https://www.cnn.com/2021/12/01/health/us-omicron-variant-confirmed-case/index.html
https://www.theguardian.com/world/2021/dec/01/omicron-covid-variant-discovered-in-west-africa-and-the-gulf
https://www.aljazeera.com/news/2021/12/1/scientists-say-vaccines-can-help-amid-omicron-variant-surge-live
https://www.reuters.com/world/us/exclusive-us-orders-airlines-disclose-passengers-who-have-been-southern-africa-2021-12-01/
https://spectrumlocalnews.com/nc/charlotte/coronavirus/2021/12/01/faq--what-to-know-about-the-new-covid-variant-in-north-carolina
https://fox11online.com/news/coronavirus/green-bay-doctor-new-covid-variant-is-not-cause-for-panic
https://www.wptv.com/news/region-c-palm-beach-county/west-palm-beach/doctor-believes-new-covid-variant-already-in-florida
https://www.f

In [5]:
summarization

[' CDC confirms first U.S. case of omicron Covid variant, in',
 ' 1st Case of Omicron COVID Variant Detected in Mass. – NBC Boston',
 " Florida's first case of omicron variant confirmed in Tampa . New Mexico's coron",
 ' Javid advised to take ‘stringent’ Covid measures within a week',
 ' Prime Minister Boris Johnson says at least one patient has died with omicron Covid']

In [6]:
import statistics

length = [len(txt.split()) for txt in summarization]
avg_len = statistics.mean(length)
summarizer = pipeline("summarization")
summ=' '.join([str(i) for i in summarization])
summarized = summarizer(summ, min_length=5, max_length=int(avg_len+5))
print(summarized[0]['summary_text'])

 CDC confirms first U.S. case of omicron Cov
